In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
import torch
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

데이터 로드

In [22]:
df1 = pd.read_csv('ndata_final_labeled.csv')
df2 = pd.read_csv('ChatGPT 생성 댓글.txt', header=None, names=['comments'], sep = '\n')
df2[['hate']]=1
df = pd.concat([df1[['comments', 'hate']], df2,])

KcBert-base

In [13]:
df1 = pd.read_csv('ndata_final_labeled.csv')
df2 = pd.read_csv('ChatGPT 생성 댓글.txt', header=None, names=['comments'], sep = '\n')
df2[['hate']]=1
df = pd.concat([df1[['comments', 'hate']], df2,])

In [25]:
#df = pd.read_csv('./ndata_final_labeled.csv')
comments = df['comments'].tolist()
labels = df['hate'].tolist()

#학습 & 검증 데이터셋 분리
train_texts, eval_texts, train_labels, eval_labels = train_test_split(comments, labels, test_size=0.2, random_state=42)

In [6]:
##KcBERT 로드
from transformers import AutoTokenizer, AutoModelWithLMHead
# Base Model (108M)
tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-base")
#model = AutoModelWithLMHead.from_pretrained("beomi/kcbert-base")

# Large Model (334M)
#tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-large")
#model = AutoModelWithLMHead.from_pretrained("beomi/kcbert-large")

In [7]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

In [27]:
def tokenize(texts, labels):
    if isinstance(texts, pd.Series):
        texts = texts.tolist()
    elif not isinstance(texts, list):
        raise ValueError("texts should be a list of strings")
    
    encodings = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    return encodings, torch.tensor(labels)  # Convert labels to a tensor


In [28]:
# Example assuming a straightforward approach
train_texts = df['comments'].astype(str).tolist()  
train_labels = df['hate'].tolist()  

eval_texts = df['comments'].astype(str).tolist() 
eval_labels = df['hate'].tolist()  

# 토큰화
train_encodings, train_labels = tokenize(train_texts, train_labels)
eval_encodings, eval_labels = tokenize(eval_texts, eval_labels)

In [29]:
import torch
from torch.utils.data import Dataset

class HateSpeechDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

# PyTorch 데이터셋 생성
train_dataset = HateSpeechDataset(train_encodings, train_labels)
eval_dataset = HateSpeechDataset(eval_encodings, eval_labels)

In [30]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# KcBERT model 로드
model = AutoModelForSequenceClassification.from_pretrained("beomi/kcbert-base", num_labels=2)

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Train the model
trainer.train()

# 모델 저장
model.save_pretrained('./kcbert_hate_speech_model')


Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly i

Epoch,Training Loss,Validation Loss
1,0.125200,0.070439
2,0.143200,0.082697
3,0.035300,0.014588


c:\users\user\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [8]:
# 학습된 모델 및 토크나이저 로드
trained_model = AutoModelForSequenceClassification.from_pretrained('./kcbert_hate_speech_model')
trained_tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-base")

# 예측 함수
def predict(text):
    inputs = trained_tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = trained_model(**inputs)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    return probabilities.detach().numpy()

In [31]:
print("\"ㅄ이니?\"", predict("ㅄ이니?"))
print("\"일본인이니?\"", predict("일본인이니?"))
print("\"페미들이랑 똑같노\"", predict("페미들이랑 똑같노"))
print("\"페미들이랑 똑같노 능지 수준 알 만하다\"", predict("페미들이랑 똑같노 능지 수준 알 만하다"))
print("\"장애인들이 세금을 좀먹는다\"", predict("장애인들이 세금을 좀먹는다"))

"ㅄ이니?" [[0.01081149 0.9891885 ]]
"일본인이니?" [[0.99678266 0.00321729]]
"페미들이랑 똑같노" [[0.99743575 0.00256424]]
"페미들이랑 똑같노 능지 수준 알 만하다" [[0.1945404 0.8054596]]
"장애인들이 세금을 좀먹는다" [[0.00139185 0.9986082 ]]


In [30]:
print(predict('너 능지가 처참한 수준이구나'))

[[0.95490354 0.04509645]]
